# Werkcollege-opdrachten Week 1.3

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales_train.sqlite in een makkelijk te vinden map

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales_train.sqlite te maken.

In [ ]:
import sqlite3 as sql
conn = sql.connect('../data/raw/go_sales_train.sqlite')

Lees van de ingelezen go_sales_train-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- returned_item
- return_reason

In [ ]:
sql_string = "SELECT * FROM sqlite_master;"
cursor = conn.cursor()

response = cursor.execute(sql_string).fetchall()
column_names = [desc[0] for desc in cursor.description]

goSalesDf = pd.DataFrame(response, columns=column_names)
print(goSalesDf)

Krijg je een "no such table" error? Dan heb je misschien met .connect() per ongeluk een leeg  databasebestand (.sqlite) aangemaakt. <u>Let op:</u> lees eventueel de informatie uit het Notebook van werkcollege 1.1b nog eens goed door.

Als je tijdens onderstaande opdrachten uit het oog verliest welke tabellen er allemaal zijn, kan je deze Pythoncode uitvoeren:

In [ ]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
pd.read_sql(sql_query, conn)
#Op de puntjes hoort de connectie naar go_sales_train óf go_staff_train óf go_crm_train te staan.

erachter 

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
response1 = pd.read_sql("SELECT PRODUCT_NAME, PRODUCTION_COST FROM product", conn)
filteredDf1 = response1.query('PRODUCTION_COST > 50 and PRODUCTION_COST < 100')

print(filteredDf1)

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
response2 = pd.read_sql("SELECT PRODUCT_NAME, MARGIN from product", conn)
filteredDf2 = response2.query('MARGIN > 0.60 or MARGIN < 0.20')

print(filteredDf2)

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
response3 = pd.read_sql("SELECT COUNTRY, CURRENCY_NAME from country", conn)
filteredDf3 = response3[response3['CURRENCY_NAME'] == 'francs']

print(filteredDf3['COUNTRY'])

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:
response4 = pd.read_sql("SELECT * from product", conn)
filteredDf4 = response4[response4['MARGIN'] > 0.50]

print(pd.DataFrame(filteredDf4['INTRODUCTION_DATE'].unique()))

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [ ]:
response5 = pd.read_sql("SELECT ADDRESS1, ADDRESS2, CITY, REGION from sales_branch", conn)
filteredDf5 = response5[response5['ADDRESS2'].notnull() & response5['REGION'].notnull()]

print(filteredDf5[['ADDRESS1', 'CITY']])

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
response6 = pd.read_sql("SELECT COUNTRY, CURRENCY_NAME from country", conn)
filteredDf6 = response6[response6['CURRENCY_NAME'].str.contains('dollar')]

print(filteredDf6)

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [88]:
response7 = pd.read_sql("SELECT ADDRESS1, ADDRESS2, CITY, POSTAL_ZONE from retailer_site", conn)
filteredDf7 = response7[response7['POSTAL_ZONE'].str.startswith('D') & response7['ADDRESS2'].notnull()]

print(filteredDf7[['ADDRESS1', 'ADDRESS2', 'CITY']])

            ADDRESS1       ADDRESS2       CITY
31  Röntgenstraße 90  3. Tür rechts  Frankfurt
59   Grubesallee 141       4. Stock    Hamburg


## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [99]:
response8 = pd.read_sql('SELECT COUNT(*) as RESPONSES FROM returned_item', conn)

print(response8)

   RESPONSES
0        619


Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [100]:
response9 = pd.read_sql('SELECT REGION FROM sales_branch', conn)
filteredDf9 = response9.count()

print(filteredDf9)

REGION    15
dtype: int64


Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [110]:
response9 = pd.read_sql('SELECT MARGIN FROM product', conn)
laagste = pd.Series(response9['MARGIN'].min())
hoogste = pd.Series(response9['MARGIN'].max())
gemiddelde = pd.Series(response9['MARGIN'].mean().round(2))

filteredDf10 = pd.concat([laagste, hoogste, gemiddelde], axis=1)
print(filteredDf10)

      0    1    2
0  0.17  0.7  0.4


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [114]:
response11 = pd.read_sql("SELECT ADDRESS2 from sales_branch", conn)
filteredDf11 = response11['ADDRESS2'].isnull().sum()

print(filteredDf11)

19


Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [119]:
response12 = pd.read_sql("SELECT UNIT_PRICE, UNIT_SALE_PRICE from order_details", conn)
filteredDf12 = response12[response12['UNIT_SALE_PRICE'] < response12['UNIT_PRICE']].sum()

print(filteredDf12)

2321026.695


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 7 rijen) 

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [ ]:
from datetime import date
date.today().year

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [ ]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

## Van Jupyter Notebook naar Pythonproject

1. Richt de map waarin jullie tot nu toe hebben gewerkt in volgens de mappenstructuur uit de slides.
2. Maak van de ontstane mappenstructuur een Pythonproject dat uitvoerbaar is vanuit de terminal. Maak daarin een .py-bestand dat minstens 5 antwoorden uit dit notebook (in de vorm van een DataFrame) exporteert naar Excelbestanden. Alle notebooks mogen als notebook blijven bestaan.
3. Zorg ervoor dat dit Pythonproject zijn eigen repo heeft op Github. Let op: je virtual environment moet <b><u>niet</u></b> meegaan naar Github.

Je mag tijdens dit proces je uit stap 1 ontstane mappenstructuur aanpassen, zolang je bij het beoordelingsmoment kan verantwoorden wat de motivatie hierachter is. De slides verplichten je dus nergens toe.